In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import time
import warnings 
warnings.filterwarnings('ignore')

## 지표 불러오기 (접어두기)

In [ ]:
# idx_lst = []
# url = 'https://finance.naver.com/item/main.naver?code=066570'
# res = requests.get(url)
# soup = BeautifulSoup(res.text,'html.parser')
# section = soup.find_all('tbody')[2]
# for n in range(8,23+1):
#     idx_lst.append(section.find('th', attrs={'class':f'h_th2 th_cop_anal{n}'}).get_text())

# idx_lst


['매출액',
 '영업이익',
 '당기순이익',
 '영업이익률',
 '순이익률',
 'ROE(지배주주)',
 '부채비율',
 '당좌비율',
 '유보율',
 'EPS(원)',
 'BPS(원)',
 '주당배당금(원)',
 'PER(배)',
 'PBR(배)',
 '시가배당률(%)',
 '배당성향(%)']

In [ ]:
def get_value(tick):
    value = []
    url = 'https://finance.naver.com/item/main.naver?code='+tick
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    section = soup.find_all('tbody')[2]

    for i in range(15+1):
        temp = section.find_all('tr')[i]
        try :
            
            if temp.find('td',attrs={" t_line cell_strong"}) == None:  # 22년 발표 안된 경우 21년도 데이터 가져옴 
                part = temp.find_all('td',attrs={""})[2]
                value.append(part.get_text().strip()+'(21)')  
            
            else:
                part = temp.find('td',attrs={" t_line cell_strong"})
                value.append(part.get_text().strip())
        except: # 값이 엾을 경우 공백처리 
            value.append('-')

        time.sleep(1)
        
    df = pd.DataFrame({'지표':idx,'value':value})
    return df

## kospi200 지표 불러오기

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/자연어프로젝트/ksp200.csv')
data

,ticker,name
0,5930,삼성전자
1,373220,LG에너지솔루션
2,660,SK하이닉스
3,207940,삼성바이오로직스
4,51910,LG화학
...,...,...
195,105630,한세실업
196,241590,화승엔터프라이즈
197,284740,쿠쿠홈시스
198,294870,HDC현대산업개발


In [ ]:
idx = ['ticker','name','매출액(억원)', '영업이익(억원)', '당기순이익(억원)', '영업이익률(%)', '순이익률(%)', 'ROE(지배주주%)', '부채비율(%)', '당좌비율(%)', '유보율(%)', 'EPS(원)','PER(배)', 'BPS(원)','PBR(배)', '주당배당금(원)',   '시가배당률(%)', '배당성향(%)']

def get_value(tick, name):
    value = [tick, name]
    url = 'https://finance.naver.com/item/main.naver?code=' + tick
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    section = soup.find_all('tbody')[2]

    for i in range(15+1):
        temp = section.find_all('tr')[i]
        try :
            part = temp.find('td',attrs={" t_line cell_strong"})
            if part == None: # 22년 발표 안된 경우 21년도 데이터 가져옴 
                part = temp.find_all('td',attrs={""})[2]
                value.append(part.get_text().strip()+'(21)')
            else:
                value.append(part.get_text().strip())
        except: # 값이 없을 경우 공백처리
            value.append('-')

        time.sleep(1)
        
    return value

def get_df(ticks):
    result = []
    for i, tick in enumerate(ticks):
        name = data.loc[i, 'name']
        result.append(get_value(tick, name))

    df = pd.DataFrame(result, columns=idx)
    return df

In [ ]:
ticks = list(data['ticker'].astype(str).str.zfill(6)) # 기업코드는 6자리여야 함.
df = get_df(ticks)

In [ ]:
df

,ticker,name,매출액(억원),영업이익(억원),당기순이익(억원),영업이익률(%),순이익률(%),ROE(지배주주%),부채비율(%),당좌비율(%),유보율(%),EPS(원),PER(배),BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%)
0,005930,삼성전자,"3,054,876","463,279","371,012",15.17,12.14,11.67,39.92(21),196.75(21),"33,143.62(21)","5,374",10.29,"48,462",1.14,"1,521",1.84(21),25.00(21)
1,373220,LG에너지솔루션,"251,891","14,273","9,004",5.67,3.57,6.50,(21),(21),(21),"3,622",122.58,"76,516",5.80,24,-,-
2,000660,SK하이닉스,"450,663","75,122","50,179",16.67,11.13,7.51,54.98(21),99.92(21),"1,643.64(21)","6,636",13.79,"96,591",0.95,"1,446",1.18(21),11.03(21)
3,207940,삼성바이오로직스,"27,487","9,093","6,102",33.08,22.20,10.26,59.68(21),80.95(21),"2,922.69(21)","8,815",90.76,"98,707",8.10,-,-,-(21)
4,051910,LG화학,"527,186","33,343","27,163",6.33,5.15,9.32,120.32(21),80.50(21),"5,311.19(21)","31,091",22.03,"389,904",1.76,"10,375",1.95(21),25.49(21)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,105630,한세실업,"22,196","1,878","1,000",8.46,4.50,19.91,161.88(21),52.73(21),"2,395.90(21)","2,510",6.72,"13,865",1.22,585,2.28(21),29.14(21)
196,241590,화승엔터프라이즈,"17,147",694,381,4.05,2.22,6.45,141.68(21),57.08(21),"1,138.81(21)",580,19.01,"9,144",1.21,55,0.27(21),-41.84(21)
197,284740,쿠쿠홈시스,"8,932(21)","1,786(21)","1,692(21)",20.00(21),18.94(21),26.53(21),38.55(21),173.48(21),"28,714.40(21)","6,780(21)",5.78(21),"28,790(21)",1.36(21),650(21),1.66(21),9.58(21)
198,294870,HDC현대산업개발,"33,639(21)","2,734(21)","1,763(21)",8.13(21),5.24(21),6.28(21),127.58(21),143.88(21),776.48(21),"2,679(21)",8.55(21),"43,665(21)",0.52(21),600(21),2.62(21),22.40(21)


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/자연어프로젝트/kospi_200.csv',index=False)

## 회사별 파일 따로 저장하기

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/자연어프로젝트/ksp200.csv")
df

,ticker,name
0,5930,삼성전자
1,373220,LG에너지솔루션
2,660,SK하이닉스
3,207940,삼성바이오로직스
4,51910,LG화학
...,...,...
195,105630,한세실업
196,241590,화승엔터프라이즈
197,284740,쿠쿠홈시스
198,294870,HDC현대산업개발


In [ ]:
idx = ['매출액', '영업이익', '당기순이익', '영업이익률', '순이익률', 'ROE(지배주주)', '부채비율', '당좌비율', '유보율', 'EPS(원)','PER(배)', 'BPS(원)','PBR(배)', '주당배당금(원)', '시가배당률(%)', '배당성향(%)']

def get_value(tick):
    value = []
    url = 'https://finance.naver.com/item/main.naver?code='+tick
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    section = soup.find_all('tbody')[2]

    for i in range(15+1):
        temp = section.find_all('tr')[i]
        try :
            if temp.find('td',attrs={" t_line cell_strong"}) == None:
                part = temp.find_all('td',attrs={""})[2]
                value.append(part.get_text().strip()+'(21)')  
            else:
                part = temp.find('td',attrs={" t_line cell_strong"})
                value.append(part.get_text().strip())
        except:
            value.append('-')
        time.sleep(1)
        
    df = pd.DataFrame({'지표':idx,'value':value})
    return df


In [ ]:
for index, row in df.iterrows():
    ticker = df['ticker'].astype(str)
    ticker = ticker.str.zfill(6)
    name = row['name']
    result = get_value(ticker[0])
    result.to_csv(f"/content/drive/MyDrive/AntHouse/0.Data/enterprise/kospi200/{name}.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
